In [3]:
%cd ../

/home/erland.fuadi/Python Project/supertrainer


In [4]:
from dotenv import load_dotenv
from hydra import compose, initialize
from omegaconf import OmegaConf

from supertrainer import StrictDict
from supertrainer.utils.helpers import login_hf

load_dotenv()
login_hf()

2024-09-18 23:09:53.834 | DEBUG    | supertrainer.utils.helpers:login_hf:40 - Use token from environment variable HUGGINGFACE_API_KEY


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/erland.fuadi/.cache/huggingface/token
Login successful
{'trainer': {'testing': False, 'model_kwargs': {'attn_implementation': 'flash_attention_2', 'torch_dtype': 'auto', 'use_cache': False, 'device_map': 'auto'}, 'bitsandbytes_kwargs': {'load_in_4bit': True, 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_compute_dtype': None}, 'training_kwargs': {'gradient_accumulation_steps': 64, 'warmup_steps': 5, 'num_train_epochs': 3, 'per_device_train_batch_size': 4, 'per_device_eval_batch_size': 1, 'learning_rate': 2e-05, 'fp16': False, 'bf16': True, 'logging_steps': 20, 'logging_strategy': 'steps', 'optim': 'adamw_8bit', 'weight_decay': 0.01, 'lr_scheduler_type': 'linear', 'seed': 3407, 'report_to': 'wandb', '

In [8]:
from pprint import pprint

with initialize(config_path="../configs", version_base=None):
    cfg = compose(config_name="train", overrides=["+experiment=train_longformer"])
    cfg = OmegaConf.to_container(cfg, resolve=True)
    cfg = StrictDict(cfg)
    pprint(cfg)

{'dataset': {'class_name': 'supertrainer.data.supertrainer.SupertrainerBERTDataset',
             'dataset_kwargs': {'path': 'masa-research/manual_absa_annotation__dgv__20240903_103',
                                'tokenizer_name_or_path': 'markussagen/xlm-roberta-longformer-base-4096'},
             'testing': False},
 'trainer': {'bitsandbytes_kwargs': {'bnb_4bit_compute_dtype': None,
                                     'bnb_4bit_quant_type': 'nf4',
                                     'load_in_4bit': True},
             'class_name': 'supertrainer.trainers.hf_trainer.HuggingFaceBERTTrainer',
             'classes': ['positif', 'negatif', 'netral'],
             'model_kwargs': {'attn_implementation': 'flash_attention_2',
                              'device_map': 'auto',
                              'torch_dtype': 'auto',
                              'use_cache': False},
             'model_name': 'markussagen/xlm-roberta-longformer-base-4096',
             'peft_kwargs': {'bi

In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
MAX_SEQUENCE_LENGTH = 4096
MODEL_NAME_OR_PATH = "markussagen/xlm-roberta-longformer-base-4096"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME_OR_PATH,
    max_length=MAX_SEQUENCE_LENGTH,
    padding="max_length",
    truncation=True
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_OR_PATH, num_labels=2)

In [9]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768

In [2]:
from datasets import load_dataset

dataset = load_dataset("masa-research/absa_news__1.5k__20240923_154258")
dataset

DatasetDict({
    train: Dataset({
        features: ['Article Reference', 'Article ID', 'Title', 'Content', 'Date', 'Article Media', 'Article Submedia', 'Journalist', 'Entity', 'Entity Sentiment', 'Difficulty', 'General Sentiment', 'Notes', 'Label'],
        num_rows: 1076
    })
    validation: Dataset({
        features: ['Article Reference', 'Article ID', 'Title', 'Content', 'Date', 'Article Media', 'Article Submedia', 'Journalist', 'Entity', 'Entity Sentiment', 'Difficulty', 'General Sentiment', 'Notes', 'Label'],
        num_rows: 62
    })
    test: Dataset({
        features: ['Article Reference', 'Article ID', 'Title', 'Content', 'Date', 'Article Media', 'Article Submedia', 'Journalist', 'Entity', 'Entity Sentiment', 'Difficulty', 'General Sentiment', 'Notes', 'Label'],
        num_rows: 216
    })
})

In [ ]:
dataset["train"]